Praktický úkol
---------------------

**V následující úloze si ukážeme praktický příklad využití družicových dat. Na základě rozdílu NDVI indexů z let 2006 a 2011, vyhodnotíme požkození lesa. Snímky jsou s výřezu Landsat 5 TM, z oblasti, kde došlo k požkození (větrný polom, a kůrovcová kalamita).**

In [ ]:
library(raster)

Nejdříve zkusíme vizuálně porovnat snímky ve viditelných RBG barvách:

In [ ]:
rgb_2006<-stack("../../data/landsat/20060831_rgb.tif")

In [ ]:
plotRGB(rgb_2006,r=3,g=2,b=1,stretch="lin")

In [ ]:
rgb_2011<-stack("../../data/landsat/20110914_rgb.tif")
plotRGB(rgb_2011,r=3,g=2,b=1,stretch="lin")

Vvtvoříme RasterStack se dvěma rastry - ndvi indexy z let 2006 a 2011. NDVI jsou vypočteny se stejných snímků jako předchozí RGB.

In [ ]:
dat_s<-stack(list.files(path="../../data/landsat/", pattern="clip",full.name=T))

In [ ]:
dat_s

In [ ]:
names(dat_s)<-c("ndvi_2006","ndvi_2011")
dat_s

**Vytvoříme rastr rozdílu NDVI**

In [ ]:
diff<-dat_s[[2]]-dat_s[[1]]
diff

In [ ]:
plot(diff)

In [ ]:
hist(diff,nclass=100)

Jednou z možností jak data reklasifikovat je vytvoření masky - rastru s hodnotami 1 a 0. Toho lze v R docílít logické operace se zadáním prahové hodnoty.

In [ ]:
diff_500 <- diff < -500
plot(diff_500)

#nebo:
#diff[diff > -1000] <- NA

Další možností je funkce `raster::reclassify`, která nám vytvoří kategorizovaný rastr:

In [ ]:
diff_reclass=reclassify(diff,c(-Inf,-500,-1,-500,1000,NA,1000,Inf,1))

In [ ]:
diff_reclass

In [ ]:
plot(diff_reclass,col=heat.colors(3))

Na výsledný rastr, můžeme použít filtr pomocí `raster::focal`

In [ ]:
diff_500_filter<-focal(diff_500,matrix(1,9,9),fun=modal)
plot(diff_500_filter)

In [ ]:
writeRaster(diff_500_filter,"damaged.tif")